# Focusing by content

In [78]:
import math
import numpy as np 
import tensorflow as tf


# Reset graph - just in case.
tf.reset_default_graph()


def focusing_by_location(keys, beta, memory):
    """Computes content addressing. Uses both address and content part for calculation of the similarity.
    Args:
        key: a 2-D Tensor [BATCH_SIZE x SLOT_SIZE] 
        beta: a 1-D Tensor - key strength [BATCH_SIZE x 1]
        memory: a 2-D Tensor [SLOT_SIZE x NUMBER_OF_SLOTS]
    """
    with tf.name_scope("focusing_by_location"):
    
        # Normalize batch - along samples.
        norm_keys = tf.nn.l2_normalize(keys,1, name="normalized_keys")
        # Normalize memory - along slots 
        norm_memory = tf.nn.l2_normalize(memory, 0)

        # Calculate cosine similarity [BATCH_SIZE x NUMBER_OF_SLOTS].
        similarity = tf.matmul(norm_keys, norm_memory, name="similarity")

        # Element-wise multiplication [BATCH_SIZE x NUMBER_OF_SLOTS]
        strengthtened_similarity = beta * similarity

        # Calculate weighting based on similarity along the "slot dimension" [BATCH_SIZE x NUMBER_OF_SLOTS].
        result = tf.nn.softmax(strengthtened_similarity, dim=1)
        return result


# test focusing
keys = tf.constant([[0.2, 0.3, 0.4],[0.1, 0.2, 0.7]], dtype=tf.float32)
print("keys=",keys)
beta= tf.constant([[100.0],[100.0]], dtype=tf.float32)

memory = tf.transpose(tf.constant([[0.2, 0.3, 0.4],[0.2, 0.3, 0.4],[0.3, 0.3, 0.3],[0.1, 0.2, 0.7]], dtype=tf.float32))
print("memory=",memory)

focus = focusing_by_location(keys, beta, memory)

# Finally - initialize all variables.
initialize_model = tf.global_variables_initializer()    
    
# Execute graph.
sess=tf.InteractiveSession()
# Initialize.
sess.run(initialize_model)
print("keys =\n",sess.run([keys]))
print("memory =\n",sess.run([memory]))
print("focus =\n",sess.run([focus]))
# [7,1,2,3,4,5,6]
sess.close()


keys= Tensor("Const:0", shape=(2, 3), dtype=float32)
memory= Tensor("transpose:0", shape=(3, 4), dtype=float32)
keys =
 [array([[ 0.2       ,  0.30000001,  0.40000001],
       [ 0.1       ,  0.2       ,  0.69999999]], dtype=float32)]
memory =
 [array([[ 0.2       ,  0.2       ,  0.30000001,  0.1       ],
       [ 0.30000001,  0.30000001,  0.30000001,  0.2       ],
       [ 0.40000001,  0.40000001,  0.30000001,  0.69999999]], dtype=float32)]
focus =
 [array([[  4.92605865e-01,   4.92605865e-01,   1.47291617e-02,
          5.90993950e-05],
       [  1.19946955e-04,   1.19946955e-04,   4.91867769e-10,
          9.99760211e-01]], dtype=float32)]


# Focusing by location

In [143]:
import math
import numpy as np 
import tensorflow as tf


# Reset graph - just in case.
tf.reset_default_graph()


def sharpening(batch, gamma):
    """Computes sharpening.
    Args:
        batch: a 2-D Tensor [BATCH_SIZE x NUMBER_OF_SLOTS] 
        gamma: a 1-D Tensor [BATCH_SIZE x 1]
    """
    EPS = 1e-30
    with tf.name_scope("sharpening"):    
        number_of_slots = int(batch.get_shape()[1])

        # Duplicate gammas - tf.tile is not working for partially unknown shape :] 
        gammas = []
        for i in range(number_of_slots):
            # Truncates gamma to 50!
            gammas.append(tf.minimum(gamma[:,0], 50))
        gammas_stacked = tf.transpose(tf.stack(gammas))
        print("gammas_stacked=", gammas_stacked)
        # Calculate powered batch [BATCH_SIZE x NUMBER_OF_SLOTS].
        powed_batch = tf.pow(batch, gammas_stacked)+EPS
        print("powed_batch=", powed_batch)

        # "Normalization" [BATCH_SIZE x NUMBER_OF_SLOTS].
        sharpened_batch = (powed_batch) / (tf.reduce_sum(powed_batch, axis=1, keep_dims=True))
        print("sharpened_batch=",sharpened_batch)

        return sharpened_batch


NUMBER_OF_SLOTS = 3
# test sharpening
v = tf.placeholder(tf.float32, shape=[None, NUMBER_OF_SLOTS])
#v = tf.constant([[0.2, 0.3, 0.4]], dtype=tf.float32)
print("v=",v)
g = tf.placeholder(tf.float32, shape=[None, 1])
#g = tf.constant([1.0], dtype=tf.float32)
print("g=",g)

sharp_v = sharpening(v, g)

# Finally - initialize all variables.
initialize_model = tf.global_variables_initializer()    

my_v = [[0.2, 0.3, 0.4],[0.0, 0.3, 0.9],[0.3, 0.3, 0.3],[0.1, 0.2, 0.7]]
init_g = np.transpose([50.0, 1.0, 10.0, 50.0])
my_g = np.reshape(init_g, [4,1])
my_feed_dict={v: my_v, g: my_g}

# Execute graph.
sess=tf.InteractiveSession()
# Initialize.
sess.run(initialize_model)
print("v =\n",sess.run([v], feed_dict=my_feed_dict))
print("sharp_v =\n",sess.run([sharp_v], feed_dict=my_feed_dict))
# [7,1,2,3,4,5,6]
sess.close()


v= Tensor("Placeholder:0", shape=(?, 3), dtype=float32)
g= Tensor("Placeholder_1:0", shape=(?, 1), dtype=float32)
gammas_stacked= Tensor("sharpening/transpose:0", shape=(?, 3), dtype=float32)
powed_batch= Tensor("sharpening/add:0", shape=(?, 3), dtype=float32)
sharpened_batch= Tensor("sharpening/truediv:0", shape=(?, 3), dtype=float32)
v =
 [array([[ 0.2       ,  0.30000001,  0.40000001],
       [ 0.        ,  0.30000001,  0.89999998],
       [ 0.30000001,  0.30000001,  0.30000001],
       [ 0.1       ,  0.2       ,  0.69999999]], dtype=float32)]
sharp_v =
 [array([[  7.88868831e-11,   5.66400843e-07,   9.99999404e-01],
       [  8.33333336e-31,   2.50000000e-01,   7.49999940e-01],
       [  3.33333313e-01,   3.33333313e-01,   3.33333313e-01],
       [  5.56030187e-23,   5.56036435e-23,   1.00000000e+00]], dtype=float32)]


# Circular convolution

In [1]:
import math
import numpy as np 
import tensorflow as tf


# Reset graph - just in case.
tf.reset_default_graph()


def circular_index(idx, size):
    if idx < 0: return size + idx
    if idx >= size : return idx - size
    else: return idx

def circular_convolution(batch, kernel):
    """Computes circular convolution.
    Args:
        batch: a 2-D Tensor [BATCH_SIZE x NUMBER_OF_SLOTS] 
        kernel: a 2-D Tensor [BATCH_SIZE x KERNEL_SIZE (e.g. 3)]
    """
    size = int(batch.get_shape()[1])
    kernel_size = int(k.get_shape()[1])
    kernel_shift = int(math.floor(kernel_size/2.0))

    kernels = []
    for i in range(size):
        # Create a list of index vectors.
        indices = [circular_index(i+j, size) for j in range(kernel_shift, -kernel_shift-1, -1)]
        # Reorganize batch according to indices. 
        reorganized_batch = tf.gather(batch, indices, axis=1)
        # Perform convolution.
        kernels.append(tf.reduce_sum(reorganized_batch * kernel, 1))
    # Sum elements lying on the same positions.
    result = tf.transpose(tf.dynamic_stitch([i for i in range(size)], kernels))
    return result

# test circular convolution
v = tf.constant([[1,2,3,4,5,6,7],[1,2,3,4,5,6,7],[0.1,0.2,0.3,0.4,0.5,0.6,0.7]], dtype=tf.float32)
print("v=",v)
k = tf.constant([[0,0,1],[0,0.5,0.5],[1,0,0]], dtype=tf.float32)
print("k=",k)

conv = circular_convolution(v, k)

# Finally - initialize all variables.
initialize_model = tf.global_variables_initializer()    
    
# Execute graph.
sess=tf.InteractiveSession()
# Initialize.
sess.run(initialize_model)
print("conv =\n",sess.run([conv]))
# [7,1,2,3,4,5,6]
sess.close()batch_size

v= Tensor("Const:0", shape=(3, 7), dtype=float32)
k= Tensor("Const_1:0", shape=(3, 3), dtype=float32)
conv =
 [array([[ 7.        ,  1.        ,  2.        ,  3.        ,  4.        ,
         5.        ,  6.        ],
       [ 4.        ,  1.5       ,  2.5       ,  3.5       ,  4.5       ,
         5.5       ,  6.5       ],
       [ 0.2       ,  0.30000001,  0.40000001,  0.5       ,  0.60000002,
         0.69999999,  0.1       ]], dtype=float32)]


#  Sharpening
## Requirements: EPS = 1e-40, gamma truncated to 100

In [96]:
import math
import numpy as np 
import tensorflow as tf


# Reset graph - just in case.
tf.reset_default_graph()



def sharpening(batch, gamma):
    """Computes sharpening.
    Args:
        batch: a 2-D Tensor [BATCH_SIZE x NUMBER_OF_SLOTS] 
        gamma: a 2-D Tensor [BATCH_SIZE x 1]
    """
    EPS = 1e-40
    
    slots = int(batch.get_shape()[1]) # NUMBER_OF_SLOTS
    
    # Calculate powered batch [BATCH_SIZE x NUMBER_OF_SLOTS].
    pows = []
    for i in range(slots):
        pows.append(tf.pow(batch[:,i], gamma))
    print("pows=",len(pows))
    powed_batch_stacked = tf.transpose(tf.stack(pows))
    #print("powed_batch_stacked=", powed_batch_stacked)
    
    # "Normalization" [BATCH_SIZE x NUMBER_OF_SLOTS].
    sharpened_batch = (powed_batch_stacked+EPS) / (tf.reduce_sum(powed_batch_stacked, axis=1, keep_dims=True)+EPS)

    return sharpened_batch


# test sharpening
v = tf.constant([[0.2, 0.3, 0.4],[0.2, 0.3, 0.4],[0.3, 0.3, 0.3],[0.1, 0.2, 0.7]], dtype=tf.float32)
#v = tf.constant([[0.2, 0.3, 0.4]], dtype=tf.float32)
print("v=",v)
g = tf.constant([1.0, 2.0, 3.0, 4.0], dtype=tf.float32)
#g = tf.constant([1.0], dtype=tf.float32)
print("g=",g)

sharp_v = sharpening(v, g)

# Finally - initialize all variables.
initialize_model = tf.global_variables_initializer()    
    
# Execute graph.
sess=tf.InteractiveSession()
# Initialize.
sess.run(initialize_model)
print("v =\n",sess.run([v]))
print("sharp_v =\n",sess.run([sharp_v]))
# [7,1,2,3,4,5,6]
sess.close()


v= Tensor("Const:0", shape=(4, 3), dtype=float32)
g= Tensor("Const_1:0", shape=(4,), dtype=float32)
pows= 3
v =
 [array([[ 0.2       ,  0.30000001,  0.40000001],
       [ 0.2       ,  0.30000001,  0.40000001],
       [ 0.30000001,  0.30000001,  0.30000001],
       [ 0.1       ,  0.2       ,  0.69999999]], dtype=float32)]
sharp_v =
 [array([[  2.22222239e-01,   3.33333343e-01,   4.44444478e-01],
       [  1.37931034e-01,   3.10344815e-01,   5.51724136e-01],
       [  3.33333343e-01,   3.33333343e-01,   3.33333343e-01],
       [  4.13564994e-04,   6.61703991e-03,   9.92969394e-01]], dtype=float32)]
